1) Data Scraping with Beautifulsoup package. Importing necessary libraries and getting the wikipedia page.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
import json
from pandas.io.json import json_normalize 

wikipage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wikipage,'lxml')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.57 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.10 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.16 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  33.61 MB/s
vincent-0.4.4- 100% |###################

2) Extracting data from the table on wikipedia page by looping through tags.

In [2]:
zip_table = soup.find('table')
column_values = zip_table.find_all('td')
elements = len(column_values)
postcode = []
borough = []
neighborhood = []

for i in range(0, elements, 3):
    postcode.append(column_values[i].text.strip())
    borough.append(column_values[i+1].text.strip())
    neighborhood.append(column_values[i+2].text.strip())

3) Build dataframe in pandas using extracted list

In [3]:
df= pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df.columns = ['Postalcode', 'Borough', 'Neighborhood']
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


4) Make adjustments to the table as required by the assignment

In [4]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.loc[df.Neighborhood == 'Not assigned', "Neighborhood"] = df.Borough
df_grp = df.groupby(['Postalcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_grp.columns = ['Postalcode', 'Borough', 'Neighborhood']
df_grp.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


5) Add Latitude and Longitude information for each postalcode from the given csv file

In [6]:
df_latlng = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlng.columns = ['Postalcode', 'Latitude', 'Longitude']
df_merge = pd.merge(df_grp, df_latlng, on=['Postalcode'], how='inner')
df_merge.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format( len(df_merge['Borough'].unique()),df_merge.shape[0]))

The dataframe has 11 boroughs and 103 neighborhoods.
